Import of libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # To standardize the data
import cvxpy as cp

Import of the HMLasso function

In [2]:
### Adapt the path "C:/Users/Kilian/Desktop/ENSAE/STATAPP" to run the cell

import sys
sys.path.insert(1, 'C:/Users/Kilian/Desktop/ENSAE/STATAPP/Projet_Statapp/pretreatment')

import file_04_HMLasso as hml

## Data downloading and separation of the dataset

Dataset containing the types of each column from data_03.csv

In [3]:
columns_types = pd.read_csv("data_03_columns_types.csv", index_col=0)
columns_types.head(3)

,Name,Type
0,HHIDPN,Cont
1,HHID,Char
2,PN,Char


Downloading the data with social and genetic variables.

In [4]:
data = pd.read_csv("data_03.csv")

C:\Users\Kilian\AppData\Local\Temp\ipykernel_12364\3952530018.py:1: DtypeWarning: Columns (2684) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_03.csv")


The column "genetic_Section_A_or_E" have mixed types, so we change its format.

In [5]:
temporary = np.where(data['genetic_Section_A_or_E'] == 'E', 1, np.where(data['genetic_Section_A_or_E'] == 'A', 0, np.nan))

In [6]:
data["genetic_Section_A_or_E"] = temporary

Now we add the health index created by t-SNE

In [7]:
tSNE_GHI = pd.read_csv("data_tSNE_GHI.csv")

We merge the t-SNE health index to the data

In [8]:
data = data.merge(tSNE_GHI, how ='left', on ='HHIDPN')

The final outcome to predict is tSNE_GHI14, so we only keep individuals who were interviewed during the last wave (14th wave)

In [9]:
data_bis = data[data['tSNE_GHI14'].notna()]

Number of individuals present in every waves.

In [10]:
tSNE_GHI[~tSNE_GHI.isnull().any(axis=1)].shape[0]

3396

We select the outcome tSNE_GHI

In [11]:
Y = data_bis[["HHIDPN"]+["tSNE_GHI" + str(i) for i in range (1,15)]]

We drop the previous health index GHIw from the data, which won't be used as outcome.
(list_columns_GHI contains the names of GHIw columns).

We drop the outcome to create the matrix X.

In [12]:
X = data_bis.drop(["GHI" + str(i) for i in range (1,15)], axis = 1)
X.drop(["tSNE_GHI" + str(i) for i in range (1,15)], axis = 1, inplace =True)

Now we split the dataset into training, validation and test sets.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=18)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=0.5, random_state = 6)

Smaller sets while coding

In [14]:
nb_test, nb_train, nb_valid = len(X_test.index)//10, len(X_train.index)//10, len(X_valid.index)//10
X_test, Y_test = X_test.iloc[:nb_test], Y_test.iloc[:nb_test]
X_train, Y_train = X_train.iloc[:nb_train], Y_train.iloc[:nb_train]
X_valid, Y_valid = X_valid.iloc[:nb_valid], Y_valid.iloc[:nb_valid]

## Machine learning

The objective here is to make a dataset where we observe if each variable exists at each wave

In [15]:
def dataset_temporal_variables (X_train,add_tSNE_GHIw):   
    temporal_variables = {}
    waves_columns = [col for col in X_train.columns if "genetic_" not in col and col[1] in "123456789"]
    for col in waves_columns:
      char = col[0] # R or H
      if col[2] in "01234":
        wave = col[1:3]
        suffix = col[3:]
      else:
        wave = col[1]
        suffix = col[2:]
      variable = char + 'w' + suffix

      if variable not in temporal_variables.keys():
        temporal_variables[variable] = np.zeros((14), dtype=bool)

      temporal_variables[variable][int(wave)-1] = True

    temporal_variables = pd.DataFrame(temporal_variables)

    # We manually add "tSNE_GHIw":
    if add_tSNE_GHIw:
        temporal_variables["tSNE_GHIw"] = np.ones((14), dtype=bool)
        waves_columns += [f"tSNE_GHI{w}" for w in range(1,15)]
        
    return (temporal_variables,waves_columns)

In [16]:
# Timeless data
def timeless_variables(X_train,waves_columns):
    non_waves_columns = [col for col in X_train.columns if col not in waves_columns]
    To_remove = ["HHIDPN","PN","HHID","RAHHIDPN"]+["INW"+str(i+1) for i in range (14)]
    for x in To_remove:
        non_waves_columns.remove(x)
    return non_waves_columns

We put the explaining variables by wave in a list of dataset Intemporal variables are put in each one of them

In [17]:
import random

def list_wave(X_train, reduced):
    (temporal_variables , waves_columns) = dataset_temporal_variables(X_train,True)
    non_waves_columns = timeless_variables(X_train,waves_columns)
    
    #Reduce number of variables to code
    if reduced:
        temporal_variables_2 = temporal_variables.iloc[:,[i for i in range(1,15)]+[-i for i in range(1,5)]]
        non_waves_columns_2 = random.choices(non_waves_columns,k=5)

        liste = [] 
        for i in range(14):
            columns_wave_i = ["HHIDPN"]+[col.replace('w', str(i+1)) for col in temporal_variables_2.T[i].index[temporal_variables_2.T[i]] if col != "tSNE_GHIw"]
            #Add the intemporal variables only to the last wave, to avoid duplicated labels issues
            if i == 13:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns_2])
            else:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i])
                
    #All the variables
    else:
        liste = []    # len = 14 
        for i in range(14):
            columns_wave_i = ["HHIDPN"]+[col.replace('w', str(i+1)) for col in temporal_variables.T[i].index[temporal_variables.T[i]] if col != "tSNE_GHIw"]
            #Intemporal variables only in the first wave, to avoid duplicated labels issues
            if i ==  13:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i + non_waves_columns])
            else:
                liste.append(X_train.loc[X_train["INW"+str(i+1)] == 1, columns_wave_i])
                
    return (liste)
    

### HMLasso selection

We start to initialize with a first lasso on the first wave.

In [87]:
def initialize_HMLasso(liste, Y_train, mu, limit):
    
    print("wave",1)
    
    scaler = StandardScaler()#(with_std=False)
    hml.ERRORS_HANDLING = "ignore"
    
    #Prepare data
    X_train1 = liste[0].drop("HHIDPN",axis=1)
    Y_train1 = Y_train.iloc[:,1]
    Y_train1.dropna(inplace =True)
    Y_train1 = Y_train1.values
    Y_train1 = (Y_train1 - np.mean(Y_train1))/np.std(Y_train1)
    
    #Standardize X_train
    X_train1 = scaler.fit_transform(X_train1)
    
    #HMLasso
    lasso = hml.HMLasso(mu)
    lasso.fit(X_train1, Y_train1)
    
    #Selection of variables
    coefficients = np.abs(lasso.beta_opt.copy())

    var_to_keep = coefficients < 10**(limit)
    var_to_keep = np.insert(var_to_keep,0,False)
    
    entire_data = liste[0]
    selected = entire_data[entire_data.columns[~var_to_keep]]
    
    return selected

function to impute missing data created when merging by mean but without touching Na values already there before the merge.

In [88]:
def Na_management(df1, df2, index):
    
    merged = df1.merge(df2, how='outer', on = index)
    
    df1_index = df1.set_index(index)
    df2_index = df2.set_index(index)
    
    merged = merged.fillna(merged.mean())
    merged = merged.set_index(index)
    
    df1_index = df1_index.fillna("NaN")
    merged.update(df1_index)
    
    df2_index = df2_index.fillna("NaN")
    merged.update(df2_index)
    
    merged = merged.replace("NaN",np.nan)
    
    merged = merged.reset_index()
    
    return merged

Function to select variables by HMLasso

In [89]:
def HMLasso_selection(X_train, Y_train, mu, limit, reduced):
    
    liste = list_wave(X_train, reduced)
    
    print("HMLasso")
    
    selected = initialize_HMLasso(liste, Y_train, mu, limit)
    
    scaler = StandardScaler()#(with_std=False)
    hml.ERRORS_HANDLING = "ignore"
    
    for i in range (1,14) :
    
        print("wave",i+1)

        var_to_select = Na_management(selected, liste[i], "HHIDPN")

        Y_train_i = Y_train.iloc[:,[0,i+1]]
        X_Y_train = var_to_select.merge(Y_train_i, how = 'left', on = "HHIDPN")

        Y_train_i = X_Y_train[f"tSNE_GHI{i+1}"]
        X_train_i = X_Y_train.drop([f"tSNE_GHI{i+1}","HHIDPN"], axis =1)

        Y_train_i = Y_train_i.fillna(Y_train_i.mean())
        Y_train_i = Y_train_i.values
        Y_train_i = (Y_train_i - np.mean(Y_train_i))/np.std(Y_train_i)

        X_train_i = scaler.fit_transform(X_train_i)

        lasso = hml.HMLasso(mu)
        lasso.fit(X_train_i, Y_train_i)

        coefficients = np.abs(lasso.beta_opt.copy())

        var_to_keep = coefficients < 10**(limit)
        var_to_keep = np.insert(var_to_keep,0,False)

        entire_data = var_to_select
        selected = entire_data[entire_data.columns[~var_to_keep]] 
    
    return selected

### Standard Lasso selection

### Within estimator

There two types of missing values, the "one-time" missing values when someone didn't awnser a question during the interview or so and the missing values when someone wasn't interviewed at all during a wave.


For the first type, we impute those missing values with the mean of the column (Nan).
(Possibility to work on another imputation method).

Then for the individuals who weren't interviewed during a wave, we replace the missing value with the temporal mean of the variable over time (NanNan)


Finally, we compute (A faire en latex) X_vague_ti = X_ti - temporal_mean(X_ti)

In [96]:
def creation_data_within(selected):
    
    #For the "one-time" missing values imputation by mean
    X_train_within = selected.fillna(selected.mean())
    
    ###For people who weren't interviewed
    # We start by adding the INWw columns to know if the individual was interviewed during the wave w
    X_train_within = X_train_within.merge(X_train[["HHIDPN"]+["INW"+str(i) for i in range(1,15)]], how ="left", on="HHIDPN")  
    
    #We recover the missing values for people who weren't interviewed during the wave w
    X_train_within = recover_missing(X_train_within)
    
    # Creation of the data set for within regression.
    (X_train_within, temporal_variables_within) = data_set_within(X_train_within)
    
    #Still Nan values in intemporal variables
    X_train_within = X_train_within.fillna(X_train_within.mean())
    
    return (X_train_within, temporal_variables_within)

This function return a dataset containing only variables concerned by the wave.

In [97]:
import re
import pickle

def get_wave(data, wave):
  """
  This function returns a smaller dataset summarizing all data for the given wave.

  Note that it also returns columns that are not relative to any wave (for instance, 'HHIDPN')
  """

  assert wave in range(1, 15)

  regex = re.compile("[0-9]+")
  wave_columns = [col for col in data.columns if (len(regex.findall(col)) == 0 or regex.findall(col)[0] == str(wave))]
  wave_data = data[wave_columns]

  return wave_data

Function to recover the missing values for people who weren't interviewed during the wave w

In [98]:
def recover_missing(X_train_within):

    X_train_within_index = X_train_within.set_index("HHIDPN")
    wave_1 = get_wave(X_train_within_index,1)
    wave_1.loc[wave_1["INW1"] == 0] = np.nan
    wave_1["INW1"].fillna(0)
    Tempo = wave_1

    for i in range(2,15):
        wave_i = get_wave(X_train_within_index, i)
        wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
        wave_i["INWw".replace('w', str(i))].fillna(0)
        Tempo = Tempo.merge(wave_i, how= "left", on = "HHIDPN")

    return Tempo.reset_index()

Get a dataframe to know which variables are in X_train_within

Function to compute (A faire en latex) X_vague_ti = X_ti - temporal_mean(X_ti)

It creates columns containing the temporal mean of a temporal variables and then replaces the Nan values (when people weren't interviewed) by this mean. Finally it creates the dataset  X_vague_ti.

In [99]:
def data_set_within (X_train_within):
    
    temporal_variables_within = dataset_temporal_variables(X_train_within, False)[0]
    
    X_within = X_train_within.copy()
    
    for col in temporal_variables_within.columns:
        index_wave = temporal_variables_within.index[temporal_variables_within[col]==1].tolist()
        names_waves = [col.replace('w', str(i+1)) for i in index_wave]
        # (~X_within[names_waves].isna()).sum(axis=1) = number of non missing values
        X_within[col+"_MEAN"] = X_within[names_waves].sum(axis=1)/(~X_within[names_waves].isna()).sum(axis=1)
        for x in names_waves:
            # Imputing the missing values by the temporal mean
            new_col = X_within[x].fillna(X_within[col+"_MEAN"])
            X_within[x] = new_col            
            #Creating the new data for within regression X_vague
            X_within[x] = X_within[x] - X_within[col+"_MEAN"]
    return (X_within, temporal_variables_within)

Now we do the same thing to Y_train but no need to impute the Nan values since the only outcome is tSNE_GHI14 (no Nan).

In [107]:
def creation_outcome_within(Y_train):

    Y_train_within = Y_train.copy()
    
    tSNE_GHI = [f"tSNE_GHI{w}" for w in range(1,15)]

    Y_train_within["tSNE_GHIw_MEAN"] = Y_train_within[tSNE_GHI].sum(axis=1)/(~Y_train_within[tSNE_GHI].isna()).sum(axis=1)
    Y_train_within["tSNE_GHI14_within"] = Y_train_within["tSNE_GHI14"] - Y_train_within["tSNE_GHIw_MEAN"]
    
    return Y_train_within

We can now proceed to the regression

In [108]:
def get_X_Y_within(X_train_within, Y_train_within, temporal_variables_within):
    
    data_regression = X_train_within.merge(Y_train_within[["HHIDPN","tSNE_GHI14_within"]], on = "HHIDPN")
    Y_regression = data_regression["tSNE_GHI14_within"]
    list_to_drop = ["HHIDPN","tSNE_GHI14_within"]+["INW"+str(i) for i in range(1,15)]+[col+"_MEAN" for col in temporal_variables_within.columns]
    X_regression = data_regression.drop(list_to_drop,axis=1)
    
    return (X_regression, Y_regression)
    

In [109]:
from sklearn.linear_model import LinearRegression

def regression(X_regression,Y_regression):
    
    modeleReg=LinearRegression()

    modeleReg.fit(X_regression,Y_regression) 
    
    print("intercept :" , modeleReg.intercept_)
    print("coefficients :", modeleReg.coef_)

    #compute R²
    print("R² :", modeleReg.score(X_regression,Y_regression))

In [110]:
def Within_estimates(X, Y, HMLasso, mu, limit, reduced):
    
    if HMLasso:
        selected = HMLasso_selection(X, Y, mu, limit, reduced)
    else:
        selected = Lasso_selection(X, Y, mu, limit, reduced)
        
    (X_within, temporal_variables_within) = creation_data_within(selected)
    Y_within = creation_outcome_within(Y_train)
    
    (X_regression, Y_regression) = get_X_Y_within(X_within, Y_within, temporal_variables_within)
    
    regression(X_regression,Y_regression)

In [111]:
Within_estimates(X_train, Y_train, HMLasso = True, mu = 250, limit = -14, reduced=True)

HMLasso
wave 1
wave 2
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -4.5464001906118986e-06. Error handled by adding 4.5464001906118986e-06 to each eigenvalue.
wave 3
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -6.804841462278468e-06. Error handled by adding 6.804841462278468e-06 to each eigenvalue.
wave 4
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -5.4432311370396063e-05. Error handled by adding 5.4432311370396063e-05 to each eigenvalue.
wave 5
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -3.871227124529014e-06. Error handled by adding 3.871227124529014e-06 to each eigenvalue.
wave 6
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -4.204490209059371e-06. Error handled by adding 4.204490209059371e-06 to each eigenvalue.
wave 7
[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -3.5785352014151535e-05. Error handled by adding 3.5785352014151535e-05 to each eigenvalue.
wave 8
[Warning] Sigma_opt is not PSD, its minimu

C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
C:\Users\Kilian\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


[Warning] Sigma_opt is not PSD, its minimum eigenvalue is -6.0465715525218893e-05. Error handled by adding 6.0465715525218893e-05 to each eigenvalue.


C:\Users\Kilian\AppData\Local\Temp\ipykernel_12364\3189160166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_1.loc[wave_1["INW1"] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_12364\3189160166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_i.loc[wave_i["INWw".replace('w', str(i))] == 0] = np.nan
C:\Users\Kilian\AppData\Local\Temp\ipykernel_12364\3189160166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

intercept : 0.37874600754707366
coefficients : [ 5.50589417e+02 -4.68667657e+10 -2.31246148e+11 -4.20900983e+10
 -1.59145777e+09  2.66696635e+08 -1.34253368e+10 -1.23099978e+10
 -1.35273251e+10 -9.38612180e+09  1.26049171e+08 -1.05024827e+10
 -2.10543624e+11  5.48133474e+02 -4.68667657e+10  1.78896438e+11
  5.82517955e+10 -1.59145798e+09  2.66696555e+08 -1.34253368e+10
 -1.23099979e+10 -1.35273251e+10 -9.38612186e+09  1.26049162e+08
 -1.05024827e+10 -2.10543624e+11  5.54649180e+02 -4.68667657e+10
  2.71101716e+09 -3.62606555e+10 -1.59145787e+09 -1.34253368e+10
 -1.23099978e+10 -1.35273250e+10 -9.38612184e+09  1.26049195e+08
 -1.05024827e+10 -2.10543624e+11 -8.04666202e+07  5.52440857e+02
 -4.68667657e+10  1.87710800e+09  4.77064335e+10 -1.59145805e+09
  2.66696469e+08 -1.34253368e+10 -1.23099978e+10 -1.35273250e+10
 -9.38612191e+09  1.26049154e+08 -1.05024827e+10 -2.10543624e+11
 -2.22820881e+07 -8.04666198e+07  4.93319143e+00  5.51129663e+02
 -4.68667657e+10  2.92582581e+09  8.4112076